## Demo of DuckDB

### Demo Goals:
##### - Avoid using pandas dataframes to load and query b/c won't work when data > memory.
##### - Take advantage of DuckDB's super fast data loading.
##### - Directly load external data into a DuckDB database, then query.
##### - Using pandas for small query results is Ok.

<span style="color:LIGHTGREEN">Import the DuckDB package</span>

In [3]:
import duckdb

<span style="color:LIGHTGREEN">Very basic test...</span>

In [10]:
duckdb.sql("SELECT 42 as k").show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    42 │
└───────┘



<span style="color:LIGHTBLUE">Because we want to keep the data we import for future use, we will create a permanent database file for it.</span>

<span style="color:LIGHTGREEN">Create a database connection...</span>

In [5]:
import duckdb

conn = duckdb.connect('taxidbdemo.db')

<span style="color:LIGHTGREEN">Load data into a DuckDB database table.</span>

In [16]:
conn.execute('''CREATE OR REPLACE TABLE taxi_data AS 
                SELECT * FROM read_csv('./src/data/dataset.csv');''') #yellow_tripdata_2025-01.parquet

We can do some things directly on our new table.

In [17]:
conn.table('taxi_data').show(max_rows = 4)

┌────────────────────┬───────────┬───────────┬─────────────────────────┬────────────────┬────────┬──────────┬──────────────────────────┬───────────────────────────┬─────────────┬───────────────────────┬─────────────────┬───────────────┬────────────────┬────────────────┬─────────────────┬───────────────┬───────────┬───────────┬────────────┬─────────────┬─────────────┬─────────────────────┬────────────────┬────────────┬──────────────────┬────────────────┬───────────────────────┬─────────────────────┬────────────────────┬───────────────────┬─────────┬──────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────┬───────┬────────────┬──────────────┬───────────────┬─────────────┬─────────────┬───────────┬─────────────┬──────────┬──────────┬──────────────────────────────┬─────────────────────────────┬───────────────────────┬─────────────────────┬───────────────────┬──────────────┬───────┬─────────────────────────┬─────────────┬────────────────────────┬───────────

We can run any sql query we want with our new table.

fetch_df() returns a pandas dataframe but make sure it can fit into memory.

In [18]:
conn.execute("SELECT * FROM taxi_data limit 4").fetchdf()

,opportunity_id,stagename,subsource,unanswered_call_counter,unserviced_age,ageing,source,Opportunity_created_date,time_taken_for_allocation,no_of_calls,...,is_session_Working_day,Session_Start_day_inweek,Weekly_sessionsdays_count,sessions_in week,Workes-reqd/min_workex,workex-reqd/workexpmax,workex- minworkexp/reqd,workex-max/reqd,workex_reqd/Age(max),workex_reqd/Age(min)
0,0060o00001WJbrYAAT,Cold,facebook,0,0,252,facebook,02-06-2021 01:23,0,2,...,0,Saturday,1,1,0.4,0.2,2.5,5,0.060606,0.071429
1,0060o00001WJcpKAAT,Cold,facebook,0,0,241,facebook,02-06-2021 05:47,0,8,...,0,Saturday,1,1,0.4,0.2,2.5,5,0.060606,0.071429
2,0060o00001WJhn0AAD,Cold,facebook,0,0,245,facebook,03-06-2021 07:47,0,2,...,0,Saturday,1,1,0.4,0.2,2.5,5,0.060606,0.071429
3,0060o00001WJlgqAAD,Cold,facebook,0,0,127,facebook,04-06-2021 05:12,0,3,...,0,Saturday,1,1,0.4,0.2,2.5,5,0.060606,0.071429


fetchall() returns all rows as list of tuples.

In [74]:
conn.execute('''SELECT VendorID, ROUND(SUM(total_amount), 2) as total 
                FROM taxi_data GROUP BY VendorID''').fetchall()

[(1, 18841261.98), (2, 60602721.27), (6, 12401.03)]

fetchdf() returns a pandas dataframe.

In [75]:
conn.execute('''SELECT VendorID, ROUND(SUM(total_amount), 2) as total 
                FROM taxi_data GROUP BY VendorID''').fetchdf()

,VendorID,total
0,1,18841261.98
1,2,60602721.27
2,6,12401.03


Extract a subset of data to use in pandas.

In [76]:
taxidf1 = conn.execute('''SELECT * FROM taxi_data WHERE VendorID = 1''').fetchdf()

In [77]:
type(taxidf1)

pandas.core.frame.DataFrame

Use the pandas dataframe.

In [78]:
taxidf1[['VendorID','trip_distance']]

,VendorID,trip_distance
0,1,1.8
1,1,4.7
2,1,1.4
3,1,0.8
4,1,4.7
...,...,...
729727,1,0.7
729728,1,1.2
729729,1,3.9
729730,1,4.0


#### DuckDB Relation Object

We can create a Relation Object in DuckDB which is a bit like a DuckDB dataframe but is not limited o your computer's memory.

Below we are using the default dattabase connection which is in memory.

In [79]:
taxi_rel = duckdb.read_parquet("yellow_tripdata_2024-01.parquet")

In [80]:
type(taxi_rel)

duckdb.duckdb.DuckDBPyRelation

### Useful Reference to functions...
https://duckdb.org/docs/api/python/reference/

In [81]:
taxi_rel.show(max_rows = 2)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        2 │ 2024-01-01 01:22:05  │ 2024-01-01 01:33:29  │ … │         17.8 │   

In [82]:
duckdb.sql("SELECT * FROM taxi_rel").show(max_rows = 5)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:03:00  │ 2024-01-01 00:09:36  │ … │        18.75 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:17:06  │ 2024-01-01 00:35:01  │ … │         31.3 │                  2.5 │         0.0 │
│        · │          ·           │          ·           │ · │           ·  │                   ·  │          ·  │
│        · │          ·           │          ·           │ · │           ·  │   

Get some summary stats on our relation.

In [83]:
taxi_rel.describe()

┌─────────┬─────────────────────┬──────────────────────┬───┬──────────────────────┬────────────────────┐
│  aggr   │      VendorID       │ tpep_pickup_datetime │ … │ congestion_surcharge │    Airport_fee     │
│ varchar │       double        │       varchar        │   │        double        │       double       │
├─────────┼─────────────────────┼──────────────────────┼───┼──────────────────────┼────────────────────┤
│ count   │           2964624.0 │ 2964624              │ … │            2824462.0 │          2824462.0 │
│ mean    │  1.7542042431013174 │ NULL                 │ … │   2.2561220508542865 │  0.141161130863152 │
│ stddev  │ 0.43259020170360285 │ NULL                 │ … │   0.8232746699398348 │ 0.4876238872392802 │
│ min     │                 1.0 │ 2002-12-31 22:59:39  │ … │                 -2.5 │              -1.75 │
│ max     │                 6.0 │ 2024-02-01 00:01:15  │ … │                  2.5 │               1.75 │
│ median  │                 2.0 │ NULL                 

In [84]:
conn.close()